# 7.h Modele językowe - BERT

In [ ]:
!python3 -m pip install transformers

### Modelowanie języka

Wykorzystamy model PolBERT do zadania na którym był wytrenowany: wypełniania luk w tekście.

In [ ]:
from transformers import *

model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1") # model zostanie pobrany na dysk
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1") # użyjemy sparowanego z nim tokenizatora
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)

#### Tokenizacja BPE

In [ ]:
import re

simple_tokenizer = re.compile(r"\w+")


mask_token = tokenizer.mask_token # token służący do modelowania języka
sent = "Chciałem przygotować spaghetti, więc poszedłem do sklepu kupić pomidory, makaron, {} i wołowinę.".format(mask_token)
print(sent, "\n")
tokenized = tokenizer(sent)

print("Indeksy tokenów: ", tokenized["input_ids"], "\n\n")

print("Liczba słów {} vs. liczba tokenów BPE {}".format(len(simple_tokenizer.findall(sent)), len(tokenized["input_ids"])))

print("Numery tokenów:")
tok_strings = tokenizer.convert_ids_to_tokens(tokenized["input_ids"])
for tok, string in zip(tokenized["input_ids"], tok_strings):
    print(tok, string)

print(tokenized)

#### Generacja predykcji

In [ ]:
for prediction in nlp(sent): # wypisujemy predykcje modelu językowego (zadanie MLM)
    print(prediction["sequence"],"\n", prediction["score"])

## Klasyfikacja tekstu

Ponownie wykorzystamy zbiór Allegro Reviews (wykorzystanych wcześniej do treningu LSTM) z benchmarku klej, zadanie to analiza wydźwięku na trzy etykiety.

Wpierw załadujemy dane.

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/lstm_data.zip?raw=true
!mv lstm_data.zip?raw=true lstm_data.zip
!unzip lstm_data.zip

In [ ]:
import re
import os

simplification = {"1.0":"1.0", # mapowanie 5-stopniowej skali ocen, na 3-stopniową
                 "2.0":"1.0",
                 "3.0":"2.0",
                 "4.0":"3.0",
                 "5.0":"3.0",}

def load_data(path, simplify=False): # simplify określa czy korzystamy z 5, czy z 3-stopniowej skali
    with open(path) as f:
        data = f.read()
    doc_pairs = data.split("\n")[:-1]
    out = []
    for doc_pair in doc_pairs[1:]:
        text, label = doc_pair.split("\t")
        if simplify:
            label = simplification[label]
        if len(simple_tokenizer.findall(text)) > 0: # odfiltrowujemy anomalie
            out.append((text, int(float(label) -1)))
    if simplify:
        num_labels = 3
    else:
        num_labels = 5
    return out, num_labels

train_data, num_labels = load_data(os.path.join("lstm_data", "train.tsv"), True)
dev_data, _ = load_data(os.path.join("lstm_data", "dev.tsv"), True)
label_list = list(range(0, num_labels))

print("Kategorie dla klasyfikatora: ", label_list, "\n")
print("Długość zbioru treningowego (liczona w dokumentach): ", len(train_data))
print("Przykładowy dokument:")
print(train_data[100])
doc_lens = [len(simple_tokenizer.findall(t)) for t, l in train_data]
print("Średnia długość dokumentu: ", sum(doc_lens)/len(doc_lens))

In [ ]:
import torch

DEVICE = torch.device("cuda:0")

#### Definicja funkcji do pracy z danymi

In [ ]:
PAD_TOKEN_ID = tokenizer.pad_token_id

def examples_to_batch(examples, maxlen):
    tokenized = tokenizer(examples) # tokenizacja tekstu
    input_ids = tokenized["input_ids"] # pobranie indeksów tokenów
    attention_mask = tokenized["attention_mask"] # pobranie maski dla uwagi
    for i, (inp, att) in enumerate(zip(input_ids, attention_mask)):
        inp_len = len(inp) # liczba tokenów
        inp = inp[:maxlen] + [PAD_TOKEN_ID] * (maxlen - inp_len) # padding wejścia
        att = att[:maxlen] + [PAD_TOKEN_ID] * (maxlen - inp_len) # padding maski
        input_ids[i], attention_mask[i] = inp, att
    X = torch.LongTensor(input_ids).to(DEVICE) # reprezentacja wejścia jako tensora liczb naturalnych
    ATT = torch.BoolTensor(attention_mask).to(DEVICE) # maska pozwalająca na ukrycie paddingu przed modelem
    return X, ATT

def train_on_batch(model, optimizer, X, ATT, Y):
    # nie aktualizujemy wag własnoręcznie, tylko korzystamy z optimizera
    model.train()
    optimizer.zero_grad()# zerujemy gradienty w optimizerze (a nie w modelu)
    output = model(input_ids=X, attention_mask=ATT, labels = Y)
    loss = output["loss"]
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # obcinanie gradientów (dla uniknięcia eksplozji gradientów)
    optimizer.step() # aktualizacja wag
    return loss.item()


def test_on_batch(model, optimizer, X, ATT, Y):
    model.eval()
    output = model(input_ids=X, attention_mask=ATT, labels = Y)
    decision = output["logits"].topk(1).indices.squeeze() # pobieramy decyzje klasyfikatora
    loss = output["loss"].item()
    equal = decision == Y # tensor określający które decyzje zostały poprawnie wydane
    correct = sum(equal).item() # sumowanie liczby poprawnych decyzji
    return correct, decision, loss
    

#### Przygotowanie przykładowego batcha

In [ ]:
examples = [e[0] for e in train_data[0:8]]
maxlen = 128
tokenized = tokenizer(examples)
input_ids = tokenized["input_ids"]
print("Reprezentacja dokumentu: ", input_ids[-1])
print("Długość dokumentu: ", len(input_ids[-1]))
attention_mask = tokenized["attention_mask"]
for i, (inp, att) in enumerate(zip(input_ids, attention_mask)):
    inp_len = len(inp)
    inp = inp[:maxlen] + [PAD_TOKEN_ID] * (maxlen - inp_len)
    att = att[:maxlen] + [PAD_TOKEN_ID] * (maxlen - inp_len)
    input_ids[i], attention_mask[i] = inp, att

print("Reprezentacja po paddingu: ", inp)
print("Maska dla paddingu: ", att)
X = torch.LongTensor(input_ids).to(DEVICE)
ATT = torch.BoolTensor(attention_mask).to(DEVICE)

print("Numery tokenów:")
tok_strings = tokenizer.convert_ids_to_tokens(inp)
for tok, string in zip(inp, tok_strings):
    print(tok, string)

#### Trening modelu

In [ ]:
import random
from tqdm.notebook import tqdm

N_LABELS = 3

model = BertForSequenceClassification.from_pretrained("dkleczek/bert-base-polish-uncased-v1",
                                                      num_labels = N_LABELS, 
                                                      #hidden_dropout_prob=0.5, 
                                                      #attention_probs_dropout_prob=0.5
                                                     )


# jeśli chcemy zamrozić parametry transformera, i modyfikować tylko klasyfikator, należy wykomentować następujące.
#for name, param in model.named_parameters():
#	  if 'classifier' not in name: # classifier layer
#		    param.requires_grad = False

model = model.to(DEVICE)

learning_rate = 0.000005 # Dla BERTa dobrze użyć niskiej wartości współczynnika uczenia

epochs = 5
batch_size = 8 # liczba przykładów we wsadzie danych, im wyższa tym szybsze uczenie, ale gorsze wyniki
maxlen = 128 # maksymalna długość przykładu
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Do aktualizacji wag wykorzystamy algorytm Adam


# obliczanie liczby batchy
num_train_batches = len(train_data) // batch_size + int(bool(len(train_data) % batch_size)) 
num_dev_batches = len(dev_data) // batch_size + int(bool(len(dev_data) % batch_size))


for epoch in range(epochs):
    #random.shuffle(train_data) # tasowanie danych
    total_loss = 0
    for n in tqdm(range(num_train_batches)): # iterujemy po numerach batchy
        datapoints = train_data[n*batch_size:(n+1)*batch_size]
        examples = [d[0] for d in datapoints]
        labels = [d[1] for d in datapoints]
        Y = torch.LongTensor(labels).to(DEVICE)
        X, ATT = examples_to_batch(examples, maxlen)
        loss = train_on_batch(model, optimizer, X, ATT, Y) # trening
        total_loss += loss
    print("train loss: ", total_loss)

    # ewaluacja na devsecie
    with torch.no_grad():
        total = 0
        correct = 0
        dev_loss = 0
        for n in range(num_dev_batches):
            datapoints = dev_data[n*batch_size:(n+1)*batch_size]
            examples = [d[0] for d in datapoints]
            labels = [d[1] for d in datapoints]
            Y = torch.LongTensor(labels).to(DEVICE)
            X, ATT = examples_to_batch(examples, maxlen)
            result, _, loss =  test_on_batch(model, optimizer, X, ATT, Y)
            dev_loss += loss
            total += batch_size
            correct += result
        acc = (correct/total) * 100
        accuracy = "{:4.2f}%".format((acc))
        print(accuracy)
        print("dev loss: ", dev_loss)
            


## Wykrywanie wynikania logicznego

To zadanie również pochodzi z benchmarka Klej. Polega na wykryciu, czy między podaną parą zdań zachodzi relacja wynikania lub sprzeczności logicznej, czy też są względem siebie neutralne.

Parę zdań przedzielimy odpowiednim separatorem, i podamy do modelu. Dodatkowo konieczne będzie podanie tensora, który będzie przechowywał informację o tym, z którego zdania pochodzi każdy z tokenów.

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/entailment.zip?raw=true
!mv entailment.zip?raw=true entailment.zip
!unzip entailment.zip

In [ ]:
from transformers import *
import re
import os
import torch

simple_tokenizer = re.compile(r"\w+")

DEVICE = torch.device("cuda:0")

def load_data(path):
    with open(path) as f:
        data = f.read()
    doc_pairs = data.split("\n")[:-1]
    out = []
    for doc_pair in doc_pairs[1:]:
        id, text1, text2, label = doc_pair.split("\t")
        out.append((text1, text2, label))
    num_labels = 3
    return out, num_labels

train_data, num_labels = load_data(os.path.join("entailment", "train.tsv"))
dev_data, _ = load_data(os.path.join("entailment", "dev.tsv"))
label_list = list(range(0, num_labels))

print("Kategorie dla klasyfikatora: ", label_list, "\n")
print("Długość zbioru treningowego (liczona w dokumentach): ", len(train_data))
print("Przykładowy dokument:")
print(train_data[100])
doc_lens = [len(simple_tokenizer.findall(t1)) for t1, t2, l in train_data]
print("Średnia długość dokumentu: ", sum(doc_lens)/len(doc_lens))

#### Sposób łączenia dwóch tekstów

In [ ]:
from transformers import *

tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1") # użyjemy sparowanego z nim tokenizatora

example = train_data[0]
tokenized = tokenizer(text=example[0], text_pair=example[1], padding=True)
converted = tokenizer.convert_ids_to_tokens(tokenized["input_ids"])
for c, tt in zip(converted, tokenized["token_type_ids"]):
    print(c, tt)

print(tokenized)

#### Funkcje do pracy z danymi

In [ ]:
PAD_TOKEN_ID = tokenizer.pad_token_id
LABEL_LIST = ["NEUTRAL", "ENTAILMENT", "CONTRADICTION"]

def examples_to_batch(examples):
    firsts = [e[0] for e in examples]
    seconds = [e[1] for e in examples]
    labels = [LABEL_LIST.index(e[2]) for e in examples] # zamiana stringów na indeksy
    tokenized = tokenizer(text=firsts, text_pair=seconds, padding=True) # tokenizacja tekstu + padding do najdłuższego w batchu
    input_ids = tokenized["input_ids"] # pobranie indeksów tokenów
    token_types = tokenized["token_type_ids"] #
    attention_mask = tokenized["attention_mask"] # pobranie maski dla uwagi
    X = torch.LongTensor(input_ids).to(DEVICE) # reprezentacja wejścia jako tensora liczb naturalnych
    ATT = torch.BoolTensor(attention_mask).to(DEVICE) # maska pozwalająca na ukrycie paddingu przed modelem
    TT = torch.LongTensor(token_types).to(DEVICE) # reprezentacja numeru zdania (pierwsze vs drugie)
    Y = torch.LongTensor(labels).to(DEVICE)
    return X, ATT, TT, Y

def train_on_batch(model, optimizer, X, ATT, TT, Y):
    # nie aktualizujemy wag własnoręcznie, tylko korzystamy z optimizera
    model.train()
    optimizer.zero_grad()# zerujemy gradienty w optimizerze (a nie w modelu)
    output = model(input_ids=X, attention_mask=ATT, token_type_ids=TT, labels = Y)
    loss = output["loss"]
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # obcinanie gradientów (dla uniknięcia eksplozji gradientów)
    optimizer.step() # aktualizacja wag
    return loss.item()


def test_on_batch(model, optimizer, X, ATT, TT, Y):
    model.eval()
    output = model(input_ids=X, attention_mask=ATT, token_type_ids=TT, labels = Y)
    decision = output["logits"].topk(1).indices.squeeze() # pobieramy decyzje klasyfikatora
    loss = output["loss"].item()
    equal = decision == Y # tensor określający które decyzje zostały poprawnie wydane
    correct = sum(equal).item() # sumowanie liczby poprawnych decyzji
    return correct, decision, loss
    

#### Trening modelu

In [ ]:
import random
from tqdm.notebook import tqdm
N_LABELS = 3

model = BertForSequenceClassification.from_pretrained("dkleczek/bert-base-polish-uncased-v1",
                                                      num_labels = N_LABELS)
model = model.to(DEVICE)

learning_rate = 2e-5#0.000005 # Dla BERTa dobrze użyć niskiej wartości współczynnika uczenia

epochs = 3
batch_size = 8 # liczba przykładów we wsadzie danych, im wyższa tym szybsze uczenie, ale gorsze wyniki
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Do aktualizacji wag wykorzystamy algorytm Adam


# obliczanie liczby batchy
num_train_batches = len(train_data) // batch_size + int(bool(len(train_data) % batch_size)) 
num_dev_batches = len(dev_data) // batch_size + int(bool(len(dev_data) % batch_size))


for epoch in range(epochs):
    #random.shuffle(train_data) # tasowanie danych
    total_loss = 0
    for n in tqdm(range(num_train_batches)): # iterujemy po numerach batchy
        examples = train_data[n*batch_size:(n+1)*batch_size]
        X, ATT, TT, Y = examples_to_batch(examples)
        loss = train_on_batch(model, optimizer, X, ATT, TT, Y) # trening
        total_loss += loss
    print("train loss: ", total_loss)

    # ewaluacja na devsecie
    with torch.no_grad():
        total = 0
        correct = 0
        dev_loss = 0
        for n in range(num_dev_batches):
            examples = dev_data[n*batch_size:(n+1)*batch_size]
            X, ATT, TT, Y = examples_to_batch(examples)
            result, _, loss =  test_on_batch(model, optimizer, X, ATT, TT, Y)
            dev_loss += loss
            total += batch_size
            correct += result
        acc = (correct/total) * 100
        accuracy = "{:4.2f}%".format((acc))
        print(accuracy)
        print("dev loss: ", dev_loss)
            
